In [ ]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

EXTRAÇÃO - Realizando a extração dos dados de arquivos csv

Para obtê-los, realizei a consulta às tabelas no dbeaver

In [ ]:
customers = pd.read_csv("/content/customers_202511281940.csv")
orders = pd.read_csv("/content/orders_202511281941.csv")
order_details = pd.read_csv("/content/order_details_202511281941.csv")
products = pd.read_csv("/content/products_202511281942.csv")

TRANSFORMAÇÃO - Realizando todas as transformações solicitadas neste bloco de código

Destacando, ao final, a realização do Merge.

In [ ]:
# Renomear colunas para snake_case
def snake(df):
    df.columns = df.columns.str.lower().str.replace(" ", "_")
    return df

customers = snake(customers)
orders = snake(orders)
order_details = snake(order_details)
products = snake(products)

# Converter datas
orders["order_date"] = pd.to_datetime(orders["order_date"])
orders["required_date"] = pd.to_datetime(orders["required_date"])
orders["shipped_date"] = pd.to_datetime(orders["shipped_date"])

# Coluna derivada: total da linha
order_details["line_total"] = (
    order_details["quantity"] * order_details["unit_price"] * (1 - order_details["discount"])
)

# Indicador: pedido atrasado
orders["late_order"] = (orders["shipped_date"] > orders["required_date"]).astype(int)

# Merge final
df_final = (
    orders
    .merge(customers, on="customer_id", how="left")
    .merge(order_details, on="order_id", how="left")
    .merge(products, on="product_id", how="left")
)


CARGA - Gerando banco Northwind DB aqui no google colab e inserindo todos os dados devidamente tratados

In [ ]:
engine = create_engine("sqlite:///northwind.db")

customers.to_sql("customers", engine, if_exists="replace", index=False)
orders.to_sql("orders", engine, if_exists="replace", index=False)
order_details.to_sql("order_details", engine, if_exists="replace", index=False)
products.to_sql("products", engine, if_exists="replace", index=False)
df_final.to_sql("orders_joined", engine, if_exists="replace", index=False)

print("Tabelas gravadas com sucesso no banco northwind.db")

Tabelas gravadas com sucesso no banco northwind.db


CONSULTAS SQL - Realizando as consultas SQL ao banco recém criado

Interessantíssimo observar como podemos gerenciar todo o fluxo de dados, transformá-lo em depois carregá-lo em uma base de dados criada aqui, no próprio Colab!

In [ ]:
conn = sqlite3.connect("northwind.db")

print("\nTop 10 clientes com mais pedidos:")
print(pd.read_sql("""
    SELECT customer_id, COUNT(*) AS total_orders
    FROM orders
    GROUP BY customer_id
    ORDER BY total_orders DESC
    LIMIT 10;
""", conn))

print("\nTop 10 produtos mais vendidos:")
print(pd.read_sql("""
    SELECT product_id, SUM(quantity) AS total_sold
    FROM order_details
    GROUP BY product_id
    ORDER BY total_sold DESC
    LIMIT 10;
""", conn))


Top 10 clientes com mais pedidos:
  customer_id  total_orders
0       SAVEA            31
1       ERNSH            30
2       QUICK            28
3       HUNGO            19
4       FOLKO            19
5       RATTC            18
6       HILAA            18
7       BERGS            18
8       BONAP            17
9       WARTH            15

Top 10 produtos mais vendidos:
   product_id  total_sold
0          60        1577
1          59        1496
2          31        1397
3          56        1263
4          16        1158
5          75        1155
6          24        1125
7          40        1103
8          62        1083
9          71        1057
